In [1]:
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk import word_tokenize as tokenize
from nltk.corpus import stopwords

In [2]:
# Load validation data
test_answer_data = pd.read_csv('C:/Users/t_p_c/AI/Term 2/ANLP/Labs/Week2/lab2resources/sentence-completion/test_answer.csv', index_col='id')
# Load testing data
testing_data = pd.read_csv('C:/Users/t_p_c/AI/Term 2/ANLP/Labs/Week2/lab2resources/sentence-completion/testing_data.csv', index_col='id')

In [3]:
# Load GoogleNews pretrained Word2Vec model
filename = r'C:\Users\t_p_c\AI\Term 2\ANLP\Labs\GoogleNews-vectors-negative300.bin.gz'
mymodel = KeyedVectors.load_word2vec_format(filename,binary=True)

In [4]:
# Split testing data in questions and answers
questions = testing_data['question']
answers = testing_data.drop(['question'],axis=1)

In [26]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\t_p_c\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\t_p_c\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
stop = stopwords.words('english')

In [5]:
# Tokenise questions for inspection
tokenised2 = []
for sentence in questions:
    tokenised2.append(tokenize(sentence))

In [6]:
# Group all missing words into list
all_unknown = []
for i, sentence in enumerate(tokenised2):
    for token in sentence:
        if token not in mymodel.vocab:
            # If a word is not in the model's vocabulary, add to list of unknowns
            all_unknown.append(token)
# Create sets of unknown words list to see one occurence of each missing word   
all_unknown_1 = set(all_unknown)
all_unknown_2 = set(all_unknown)
print(all_unknown_1)

{'worm-eaten', 'left-hand', '.', '90', 'old-fashioned', 'trap-door', 'travelled', "'Oh", 'top-hat', 'gold-mine', 'and', 'good-day', 'coat-sleeve', 'whipcord', 'centred', '250', 'battle-axe', '30,000', 'well-grown', 'manor-house', 'neighbours', 'coal-black', 'programme', 'discoloured', 'dining-room', 'ecarte', 'hansoms', 'police-station', 'bell-pull', 'ill-used', 'favour', 'dressing-gown', 'country-houses', 'frock-coat', 'to', 'fellow-countryman', "'ye", 'smooth-faced', 'half-pay', 'panelled', 'drawing-room', 'dressing-room', 'labour', 'red-headed', 'sick-room', 'station-master', 'morning-room', 'broad-shouldered', 'sitting-room', 'to-night', "'s", 'four-wheeled', 'realised', 'consulting-room', 'grey', 'ill-health', 'billiard-room', 'a', '2,000', 'equalled', 'waiting-room', 'bell-rope', 'country-side', 'door-key', 'practised', 'quarter-past', 'centre', 'dark-lantern', 'sitting-rooms', 'saviour', 'twenty-four', "'", 'high-road', 'window-sill', 'ever-changing', 'dog-cart', 'to-day', 'of',

In [7]:
# Create list of all out-of-vocab words which have a titled form in vocab
titled_words = []
remove_words = []
for word in all_unknown_1:
    if word.title() in mymodel.vocab and word not in stop:
        titled_words.append((word, word.title()))
        remove_words.append(word)
print(titled_words)
print(len(titled_words))

NameError: name 'stop' is not defined

In [8]:
# Create new sentence with all OOV words now titled
tokenised_questions = tokenised2.copy()
for sentence in tokenised_questions:
  for i, word in enumerate(sentence):
    if word not in mymodel.vocab and word not in stop:
      if word.title() in mymodel.vocab:
        sentence[i] = word.title()
# print(tokenised3)

NameError: name 'stop' is not defined

In [9]:
# Count how many composition words occur in hyphenated form, in questions only
hyph_words = []
fully_hyphenated = []
remove_words2 = []
for word in all_unknown_1:
    if '-' in word:
        fully_hyphenated.append(word)
        # Store all split hyphenated words in seperate list
        hyph_words += [word.split('-')]

In [10]:
# Filter out and remove all individual component words which are OOV
hyph_words2 = []
for item in hyph_words:
    same_hyph = []
    for word in item:
        if word in mymodel.vocab:
            same_hyph.append(word)
    hyph_words2.append(same_hyph)

In [11]:
# Compute sum of individual component word vectors
word_vectors = [] 
for words in hyph_words2:
    vectors = np.zeros([300])
    for word in words:
        vectors += mymodel.get_vector(word)
    word_vectors.append(vectors)

In [13]:
# Add hyphenated words and their new vectors to model vocab
# Incredibly time draining and computationally expensive
# Takes a lot of memory and disk, WILL NOT WORK ON COLAB
for i, word in enumerate(fully_hyphenated):
    mymodel.add(word,word_vectors[i],replace=False)

In [23]:
# NOW REPEAT THE PROCESS FOR THE ANSWERS
unknown_ans = []
for ind, q in enumerate(questions,start=1):
    for i, word in enumerate(answers.loc[ind]):
        if word not in mymodel.vocab:
            unknown_ans.append(word)
print(unknown_ans)

['country-dance', 'rumours', 'worn-out', 'panelling', 'tissue-paper', 'blood-stained', 'realised', "comin'", 'practised', 'marvelling', 'analyse', 'aeroplane', 'pocket-book', 'sepulchre', 'theatres', 'note-book', 'honour', 'motor-car', 'offence', 'good-natured', 'well-spoken', 'befel', 'first-class', 'good-hearted', 'dressing-table', 'wicker-work', 'aeroplane', 'horror-stricken', 'battle-cry', 'walking-stick', 'out-of-the-way', 'honour', "family's", 'thirty-five', 'small-pox', "king's", 'good-humoured', 'splendour', 'first-fruits', 'fresh-water', 'business-like', 'elm-tree', 'unfrequent', 'honour', 'recognising', 'fervour', 'water-side', "wit's", 'well-merited', 'honour', 'double-edged', 'organised', 'half-past', 'labouring', 'sharp-eyed', 'long-legged', 'wide-awake', 'centre', 'spoilt', 'night-gown', 'wood-work', 'grown-up', 'pocket-book', 'half-witted', 'butt-end', "lady's", 'marvellous', 'humoured', 'pyjamas', 'colour', 'intentness', 'cudgeled', "sister's", 'seven-thirty', 'ape-man'

In [24]:
hyph_ans = []
fully_hyph_ans = []
remove_words2 = []
for word in unknown_ans:
    if '-' in word:
        fully_hyph_ans.append(word)
        hyph_ans += [word.split('-')]
print(hyph_ans)

[['country', 'dance'], ['worn', 'out'], ['tissue', 'paper'], ['blood', 'stained'], ['pocket', 'book'], ['note', 'book'], ['motor', 'car'], ['good', 'natured'], ['well', 'spoken'], ['first', 'class'], ['good', 'hearted'], ['dressing', 'table'], ['wicker', 'work'], ['horror', 'stricken'], ['battle', 'cry'], ['walking', 'stick'], ['out', 'of', 'the', 'way'], ['thirty', 'five'], ['small', 'pox'], ['good', 'humoured'], ['first', 'fruits'], ['fresh', 'water'], ['business', 'like'], ['elm', 'tree'], ['water', 'side'], ['well', 'merited'], ['double', 'edged'], ['half', 'past'], ['sharp', 'eyed'], ['long', 'legged'], ['wide', 'awake'], ['night', 'gown'], ['wood', 'work'], ['grown', 'up'], ['pocket', 'book'], ['half', 'witted'], ['butt', 'end'], ['seven', 'thirty'], ['ape', 'man'], ['summer', 'time'], ['paving', 'stones'], ['long', 'standing'], ['snow', 'flake'], ['cast', 'iron'], ['to', 'morrow'], ['slow', 'witted'], ['claw', 'like'], ['store', 'room'], ['good', 'night'], ['to', 'morrow'], ['lu

In [25]:
counter2 = 0
for words in hyph_ans:
    for word in words:
        if word not in mymodel.vocab:
            counter2+=1
            
            words.remove(word)
print(hyph_ans)

[['country', 'dance'], ['worn', 'out'], ['tissue', 'paper'], ['blood', 'stained'], ['pocket', 'book'], ['note', 'book'], ['motor', 'car'], ['good', 'natured'], ['well', 'spoken'], ['first', 'class'], ['good', 'hearted'], ['dressing', 'table'], ['wicker', 'work'], ['horror', 'stricken'], ['battle', 'cry'], ['walking', 'stick'], ['out', 'the', 'way'], ['thirty', 'five'], ['small', 'pox'], ['good'], ['first', 'fruits'], ['fresh', 'water'], ['business', 'like'], ['elm', 'tree'], ['water', 'side'], ['well', 'merited'], ['double', 'edged'], ['half', 'past'], ['sharp', 'eyed'], ['long', 'legged'], ['wide', 'awake'], ['night', 'gown'], ['wood', 'work'], ['grown', 'up'], ['pocket', 'book'], ['half', 'witted'], ['butt', 'end'], ['seven', 'thirty'], ['ape', 'man'], ['summer', 'time'], ['paving', 'stones'], ['long', 'standing'], ['snow', 'flake'], ['cast', 'iron'], ['morrow'], ['slow', 'witted'], ['claw', 'like'], ['store', 'room'], ['good', 'night'], ['morrow'], ['luncheon', 'table'], ['sister', 

In [27]:
ans_vectors = [] 
for words in hyph_ans:
    vectors_2 = np.zeros([300])
    for word in words:
        vectors_2 += mymodel.get_vector(word)
    ans_vectors.append(vectors_2)
print(ans_vectors)

[array([ 1.03515625e-01, -1.84570312e-01,  3.48632812e-01,  2.97851562e-01,
        9.52148438e-03, -3.91845703e-02, -1.52099609e-01, -3.76953125e-01,
       -1.91406250e-01,  4.54101562e-02,  8.30078125e-03, -4.24804688e-01,
        2.55126953e-01,  1.23046875e-01, -1.61621094e-01,  3.84765625e-01,
       -7.08007812e-02,  7.91015625e-02,  5.95703125e-02, -2.44140625e-01,
        3.32519531e-01,  1.88743591e-01,  6.84814453e-02, -2.22656250e-01,
       -2.19726562e-02,  7.36083984e-02, -4.11132812e-01, -3.31054688e-01,
        2.12646484e-01, -6.64062500e-02, -3.41796875e-03, -2.68554688e-02,
        1.01928711e-01,  2.70507812e-01,  3.51562500e-02, -4.65576172e-01,
       -1.75781250e-02, -1.98242188e-01,  2.35351562e-01,  1.47949219e-01,
       -2.90039062e-01, -2.12402344e-01,  2.43652344e-01, -5.41992188e-02,
        1.00708008e-01,  2.21191406e-01, -2.61230469e-01, -1.30859375e-01,
        2.50366211e-01,  3.01345825e-01,  1.25000000e-01, -4.35791016e-02,
        9.69238281e-02, 

In [28]:
# Add hyphenated words and their new vectors to model vocab
# Incredibly time draining and computationally expensive
# Takes a lot of memory and disk, WILL NOT WORK ON COLAB
for i, word in enumerate(fully_hyph_ans):
    mymodel.add(word,ans_vectors[i],replace=False)
print('True')

True


In [ ]:
def getSim(model, question, answer):
    """
        Function to calculate similarity between answer words and question sentences, uses Word2Vec similarity method
        :Param model: Word2Vec model used
        :Param question: question sentence
        :Param answer: possible answer word
        :Return average_score: int representing average similarity 
    """
    # Set similarity score to be
    sim_score = 0
    # Loop through question to get similarity with answer
    for word in question:
        # Increment similarity score with Word2Vec similarity function
        sim_score += model.similarity(word, answer)
    # Get average score for question and answer
    average_score = sim_score / len(question)
    return average_score

In [ ]:
def checkQuestion(model, question):
    """
        Function to tokenise question and see if tokens are in model vocab and remove those that aren't
        :Param model: Word2Vec model to be used
        :Param question: question sentence to be checked
        :Return sentence: list of tokenised question with out-of-vocab words removed
    """
    # List containing punctuation to remove from questions
    punc = ['.', ',', '?', '!', "'", '_____']
    # Call NLTK method to tokenise sentence
    tokenised = tokenize(question)
    # List to add words which are present in the vocab
    sentence = []
    # For loop to check each word in order
    for word in tokenised:
        # If the word is punctuation, ignore it
        if word in punc:
            continue
        # If the word in the vocab add it to sentence
        elif word in model.vocab:
            sentence.append(word)
            
    return sentence

In [16]:
def checkAnswer(model, answer):
    """
        Function to check if answer in vocab
        :Param model: model to be used
        :Param answer: answer word
        :Return True: boolean
        :Return False: boolean
    """
    if answer in model.vocab:
        # If answer is in model vocab, return True
        return True
    else:
        # If answer is not in model vocab, return False
        return False

In [17]:
def maxSim(model, question, answers):
    """
        Function pass questions through checkQuestion(), then pass answers through checkAnswer()
            and get similarity between answers and corresponding question, then select answer with mx sim
        :Param model: Word2Vec model to be used
        :Param question: current question sentence
        :Param answers: possible answers associated with question
        :Return final_ans[0]: character, answer letter
    """
    # Possible answer letters
    letters = ['a', 'b', 'c', 'd', 'e']
    # Instantiate list to store each answers similarity score
    answer_scores = []
    # Call checkQuestion() to remove words not in vocab from question
    sentence = checkQuestion(model, question)
    # Enumerate through the answers, use index to choose corresponding answer letter
    for i, answer in enumerate(answers):
        # If answer is in the vocab, calculate and store average similarity with question
        if checkAnswer(model, answer):
            answer_scores.append((letters[i], getSim(model, sentence, answer)))
        # If the answer is not in the vocab, assign similarity of zero
        else:
            answer_scores.append((letters[i], 0))
    # Use max() function to choose highest scoring answer
    final_ans = max(answer_scores, key=lambda tup:tup[1])

    return final_ans[0]

In [18]:
def scoreAllQuestions(model, questions, answers):
    """
        Function to call maxSim on all quesions and store each answer
        :Param model: Word2Vec model to be used
        :Param question: current question sentence
        :Param answers: possible answers associated with question
        :Return scores: List of answers
    """
    # List to store answer letters
    scores = []
    # Enumerate through questions
    for i, q in enumerate(questions,start=1):
        # Call maxSim to obtain selected answer and store in scores
        scores.append(maxSim(model, q, answers.loc[i]))
    return scores

In [29]:
final_answers_1 = scoreAllQuestions(mymodel, tokenised_questions, answers)
final_answers_1_df = pd.DataFrame(final_answers_1, index=testing_data.index, columns=['answer'] )

In [20]:
def evaluate(scores, goldStandard):
    """Evaluation function to compare answers model obtains with validation data.
        :Return float: percentage correct
    """
    return ((scores == goldStandard).sum()/len(scores)) * 100

In [30]:
score = evaluate(final_answers_1_df, test_answer_data)
print(score)

answer    36.538462
dtype: float64
